In [ ]:
#Importing all the libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset,DataLoader
from torchvision import models,datasets,transforms

from tqdm import tqdm
import os
from PIL import Image
import matplotlib.pyplot as plt
import math
import random

In [ ]:
#Checking if a GPU with CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
#Creating a custom dataset class that generates the noisy counterpart of the clean image and returns both of them
class CustomDataset(Dataset):
  def __init__(self,train_flag,sigma):
    """
    train_flag is True for the Train dataset and False for the evaluation dataset
    sigma is the noise level
    """
    super().__init__()
    #Defining the transforms for the train and test datasets
    train_transform = transforms.Compose([transforms.RandomRotation(45),
                                              transforms.RandomHorizontalFlip(.5),
                                              transforms.RandomVerticalFlip(.5),
                                              transforms.ToTensor()
                                              ])

    test_transform = transforms.Compose([transforms.ToTensor()])

    #Downloading the train and test datasets
    if train_flag:
      self.dataset = datasets.MNIST(root="", train = True, download=True, transform=train_transform)

    else:
      self.dataset = datasets.MNIST(root="", train = False, download=True, transform=test_transform)

    self.sigma = sigma

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    clean_img, _ = self.dataset[index]
    #Generating the noisy image
    noisy_img = (torch.randn(clean_img.shape)*(self.sigma/255.))+clean_img
    return clean_img, noisy_img



In [ ]:
#Defining a Block of the DnCNN
class Block(nn.Module):
  def __init__(self,k=3,p=1,c=64):
    super().__init__()
    self.conv = nn.Conv2d(in_channels=c,out_channels=c,kernel_size=k,padding=p,bias=False) #same padding convolution
    self.norm = nn.BatchNorm2d(c) #batch normalization
    self.relu = nn.ReLU(inplace=True) #activation function

  def forward(self,x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.relu(x)
    return x


In [ ]:
#Defining the DnCNN model
class DCNN(nn.Module):
  def __init__(self,k=3,p=1,c=64,l=17,in_c=1):
    super().__init__()
    layers = [nn.Conv2d(in_channels=in_c,out_channels=c,kernel_size=k,padding=p,bias=False), #First same padding convolution layer
              nn.ReLU(inplace=True)]
    layers.extend([Block(k,p,c) for _ in range(l-2)]) #Adding all the "Blocks" to the model
    layers.append(nn.Conv2d(in_channels=c,out_channels=in_c,kernel_size=k,padding=p,bias=False)) #Last same padding convolution layer

    self.all = nn.Sequential(*layers)

  def forward(self,x):
    out = self.all(x)
    return x-out


In [ ]:
#Function for unsupervised training of the model
def train(criterion,optimizer,model,device,train_loader,sigma,alpha):
  """
  criterion is the loss function
  optimizer is the optimization algorithm used
  model is the denoiser model
  device is either CPU or GPU(cuda)
  train_loader is the DataLoader containing the training dataset
  sigma is the noise level
  alpha is the constant of 0.5
  """
  model.train()
  loop = tqdm(train_loader) #Used to visualized the progress in training
  cur_loss = 0.0

  for i,(_,noisy) in enumerate(loop): #iterating batch-by-batch through the dataset
    noisy = noisy.to(device) #Moving over the data to the "device"

    #Generating pair of noisy images from the noisy image
    noise = (torch.randn(noisy.shape)*(sigma/255.)).to(device)
    noisy1 = noisy+alpha*noise
    noisy2 = noisy-noise/alpha

    noisy2_pred = model(noisy1) #Passing the data through the model
    loss = criterion(noisy2_pred,noisy2) #Computing the loss

    optimizer.zero_grad() #Zeroing all the previous gradients
    loss.backward() #Computing the gradients for the current iteration
    optimizer.step() #Updating the weights of the model

    cur_loss += loss.item() #Keeping track of the loss
    loop.set_postfix(loss=cur_loss/(i+1)) #Printing the cumulative loss after each iteration

In [ ]:
#Function for testing the model
def test(criterion,model,device,test_loader,alpha,sigma,T):
  """
  criterion is the function used to compute PSNR
  model is the denoiser model
  device is either CPU or GPU(cuda)
  test_loader is the DataLoader containing the test dataset
  sigma is the noise level
  alpha is the constant of 0.5
  T is the number of forward processes averaged to reduce the effect of recorruption
  """

  model.eval()
  loop = tqdm(test_loader) #Used to visualized the progress in testing
  total_mse = []

  with torch.no_grad(): #Ensures that the gradients are not computed
    for i,(clean,noisy) in enumerate(loop): #iterating batch-by-batch through the dataset
      clean,noisy = clean.to(device).float(), noisy.to(device).float() #Moving over the data to the "device"

      #Averaging T forward passes
      out = torch.zeros(clean.shape).to(device)
      for _ in range(T):
        noise = (torch.randn(noisy.shape)*(sigma/255.)).to(device)
        noisy_main = noisy+alpha*noise
        out += model(noisy_main) #Passing the data through the model

      clean_pred = torch.clamp(out/T,min=0.0,max=1.0) #Clips all the values greater than 1 or less than 0
      loss = (criterion(clean_pred,clean).mean(axis=(1,2,3))).tolist() #Computing MSE at an image level
      total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean() #Computing the PSNR using the corresponding MSE values

  print(f"The PSNR is {psnr}")
  return psnr.item()

In [ ]:
#Function to computer the number of parameters in a model
def number_of_parameters(model):
    return sum(params.numel() for params in model.parameters() if params.requires_grad)

In [ ]:
#Wrapper function to train and evaluate the denoiser model
def wrapper(sigma):
  print(f"This is for sigma of {sigma}")

  #Defines the loaders for the train and test set
  train_set = CustomDataset(train_flag=True,sigma=sigma)
  test_set = CustomDataset(train_flag=False,sigma=sigma)

  train_loader = DataLoader(train_set,batch_size=128,shuffle=True,num_workers=128)
  test_loader = DataLoader(test_set,batch_size=128,shuffle=False,num_workers=128)

  print(f"The number of images in the train set is {len(train_set)}")
  print(f"The number of images in the test set is {(len(test_set))}")

  #Defining the model, loss function and optimizer
  model = DCNN().to(device)
  criterion_train = nn.MSELoss()
  criterion_test = nn.MSELoss(reduce=False)
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  epochs = 5
  alpha = 0.5
  T = 50

  print(f"The model has {number_of_parameters(model)} parameters")
  #Computing the PSNR between the noisy and clean image
  total_mse = []
  with torch.no_grad():
      for i,(clean,noisy) in enumerate(test_loader):
        clean,noisy = clean.to(device), noisy.to(device)
        loss = (criterion_test(noisy,clean).mean(axis=(1,2,3))).tolist()
        total_mse.extend(loss)

  total_mse_tensor = torch.tensor(total_mse)
  psnr = (-10*torch.log10(total_mse_tensor)).mean()
  print(f"The PSNR for an untrained densoiser is {psnr}")

  #Iterating through the epochs
  for epoch in range(epochs):
    print(f"The current epoch is {epoch}")
    train(criterion_train,optimizer,model,device,train_loader,sigma,alpha)
    cur_psnr = test(criterion_test,model,device,test_loader,alpha,sigma,T)
    torch.save(model.state_dict(), "Unsupervised"+str(epoch)+"_"+str(round(cur_psnr,2))+"_"+ str(sigma) + ".pt")


In [ ]:
#For sigma value of 10
wrapper(10)

This is for sigma of 10
The number of images in the train set is 60000
The number of images in the test set is 10000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 128 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The model has 556032 parameters


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


The PSNR for an untrained densoiser is 28.133331298828125
The current epoch is 0


...

The PSNR is 35.44024658203125
The current epoch is 3


100%|██████████| 79/79 [00:40<00:00,  1.95it/s]


The PSNR is 35.724117279052734
The current epoch is 4


100%|██████████| 79/79 [00:40<00:00,  1.93it/s]

The PSNR is 35.98470687866211


In [ ]:
#For sigma value of 25
wrapper(25)

This is for sigma of 25
The number of images in the train set is 60000
The number of images in the test set is 10000
The model has 556032 parameters
The PSNR for an untrained densoiser is 20.177610397338867
The current epoch is 0


100%|██████████| 79/79 [00:40<00:00,  1.94it/s]


The PSNR is 28.94173812866211
The current epoch is 1


100%|██████████| 79/79 [00:41<00:00,  1.91it/s]


The PSNR is 29.132030487060547
The current epoch is 2


...

The PSNR is 29.17801856994629
The current epoch is 3


100%|██████████| 79/79 [00:43<00:00,  1.80it/s]


The PSNR is 29.554399490356445
The current epoch is 4


100%|██████████| 79/79 [00:44<00:00,  1.79it/s]

The PSNR is 29.956275939941406


In [ ]:
#For sigma value of 50
wrapper(50)

This is for sigma of 50
The number of images in the train set is 60000
The number of images in the test set is 10000
The model has 556032 parameters
The PSNR for an untrained densoiser is 14.155823707580566
The current epoch is 0


100%|██████████| 79/79 [00:44<00:00,  1.78it/s]


The PSNR is 24.43431282043457
The current epoch is 1


100%|██████████| 79/79 [00:43<00:00,  1.80it/s]


The PSNR is 25.103973388671875
The current epoch is 2


...

The PSNR is 24.940763473510742
The current epoch is 3


100%|██████████| 79/79 [00:44<00:00,  1.77it/s]


The PSNR is 25.41838836669922
The current epoch is 4


100%|██████████| 79/79 [00:43<00:00,  1.80it/s]

The PSNR is 25.394750595092773
